In [20]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import snowflake.connector
import os
from dotenv import load_dotenv
from mlxtend.frequent_patterns import apriori, association_rules


load_dotenv()

True

In [21]:
# Se establece la connecion con la base de datos de snowflake
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("DESTINATION_SNOWFLAKECREDENTIALS_WAREHOUSE"),
    database=os.getenv("DESTINATION_SNOWFLAKECREDENTIALS_DATABASE_GOLD"),
    schema=os.getenv("DESTINATION_SNOWFLAKECREDENTIALS_SCHEMA")
)

In [22]:
# Se crea un cursor para ejecutar consultas
cursor = conn.cursor()

In [23]:
# Se ejecuta una consulta
cursor.execute("SELECT * FROM INVOICE_GOLD")

In [24]:
# Se obtienen todos los resultados de la consulta en un DataFrame de pandas
df = cursor.fetch_pandas_all()

In [25]:
# Se guarda el DataFrame en un archivo Parquet local
df.to_parquet(path='../data/output_file.parquet', engine='pyarrow', index=False)

In [26]:
# Cerrar el cursor y la conexion
cursor.close()
conn.close()

In [27]:
# Se lee el archivo parquet
dataframe_retail_data = pd.read_parquet('../data/output_file.parquet', engine='pyarrow')

In [28]:
# Se muestran las primeras filas del DataFrame
dataframe_retail_data.head()

,INVOICENO,STOCKCODE,DESCRIPTION,QUANTITY,UNITPRICE,COUNTRY,FECHA_INVOICE,HORA_MINUTO_VARCHAR,FECHA_DE_CARGA,HORA_DE_CARGA
0,550124,22139,RETROSPOT TEA SET CERAMIC 11 PC,6,4.95,United Kingdom,2011-04-14,12:41,2024-08-10,04:33
1,550124,22904,CALENDAR PAPER CUT DESIGN,6,2.95,United Kingdom,2011-04-14,12:41,2024-08-10,04:33
2,550124,22423,REGENCY CAKESTAND 3 TIER,4,12.75,United Kingdom,2011-04-14,12:41,2024-08-10,04:33
3,550124,84988,SET OF 72 PINK HEART PAPER DOILIES,12,1.45,United Kingdom,2011-04-14,12:41,2024-08-10,04:33
4,550124,21212,PACK OF 72 RETROSPOT CAKE CASES,24,0.55,United Kingdom,2011-04-14,12:41,2024-08-10,04:33


# Filtrado Colaborativo Basado en Ítems (Item-Based Collaborative Filtering)

### Descripcion
Este enfoque no requiere de calificaciones explícitas. En su lugar, utiliza las interacciones de los usuarios con los productos (por ejemplo, compras) para recomendar ítems similares a los que el usuario ya ha comprado

### Implementación:

### Paso 1:
Crear una matriz de co-ocurrencia de productos. Por ejemplo, cuenta cuántas veces dos productos diferentes han sido comprados juntos en la misma factura

### Paso 2:
Calcula la similitud entre productos utilizando medidas como la similitud de coseno

### Paso 3:
Para un producto dado, recomienda productos que tienen alta similitud con él según la matriz de co-ocurrencia

In [29]:
# Se crear una tabla de co-ocurrencia
co_ocurrence_matrix = dataframe_retail_data.pivot_table(index="INVOICENO", columns="STOCKCODE", aggfunc="size", fill_value=0)

In [30]:
co_ocurrence_matrix

STOCKCODE,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
INVOICENO,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A563185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# Se calculan similitudes entre productos
product_similarities = cosine_similarity(co_ocurrence_matrix.T)

In [32]:
product_similarities

array([[1.        , 0.        , 0.04138029, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.10206207,
        0.        ],
       [0.04138029, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.10206207, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [33]:
# Función para recomendar productos similares
def recommended_products(stockcode, product_similarities, n=10):
    product_idx = co_ocurrence_matrix.columns.get_loc(stockcode)
    sim_scores = list(enumerate(product_similarities[product_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n+1]
    similar_products = [co_ocurrence_matrix.columns[i[0]] for i in sim_scores]
    return similar_products

In [34]:
# Ejemplo de uso
recommendations = recommended_products(stockcode="21937", product_similarities=product_similarities)
print(recommendations)

['21936', '21932', '21933', '20725', '22384', '85099B', '85099F', '22386', '21931', '20723']


# Recomendaciones Basadas en Reglas de Asociación (Market Basket Analysis)

### Descripcion
Este enfoque analiza qué productos suelen comprarse juntos. A partir de esto, se pueden recomendar productos adicionales cuando un usaurio compra un determinado ítem

### Implementación

### Paso 1:
Utiliza el algoritmo Apriori para descubrir reglas de asociación en el dataset

### Paso 2:
Utiliza estas reglas para hacer recomendaciones de productos que suelen comprarse junto con los productos que un usuario ha adquirido

In [35]:
# Se transforma el DataFrame en un formato de cesta de compras
basket = dataframe_retail_data.groupby(["INVOICENO", "STOCKCODE"])["QUANTITY"].sum().unstack().fillna(0)

In [36]:
basket

STOCKCODE,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
INVOICENO,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A563185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

/tmp/ipykernel_9322/3498954818.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


In [38]:
basket

STOCKCODE,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
INVOICENO,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A563185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
from mlxtend.frequent_patterns import fpgrowth

# Usar FP-Growth en lugar de Apriori
# Funciona mendiante la construcción de una estructura arborescente 
# denominada árbol FP, que codifica los conjuntos frecuentes del conjunto de datos
frequent_itemsets = fpgrowth(basket, min_support=0.01, use_colnames=True)


/home/ulises/developer/mlops_wizeline_data/modelo_recomendador/venv/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [40]:
# Se generan las reglas de sociacion
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [41]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(22865),(22633),0.029686,0.021710,0.010696,0.360305,16.596612,0.010052,1.529308,0.968498
1,(22633),(22865),0.021710,0.029686,0.010696,0.492693,16.596612,0.010052,1.912676,0.960601
2,(85123A),(84879),0.099846,0.065945,0.014685,0.147072,2.230241,0.008100,1.095117,0.612804
3,(84879),(85123A),0.065945,0.099846,0.014685,0.222680,2.230241,0.008100,1.158023,0.590562
4,(84879),(22423),0.065945,0.090147,0.012237,0.185567,2.058497,0.006292,1.117161,0.550512
...,...,...,...,...,...,...,...,...,...,...
2501,(23295),(23294),0.020123,0.016044,0.010651,0.529279,32.988751,0.010328,2.090318,0.989601
2502,(23356),(23355),0.021574,0.035442,0.010696,0.495798,13.988867,0.009932,1.913039,0.948988
2503,(23355),(23356),0.035442,0.021574,0.010696,0.301790,13.988867,0.009932,1.401336,0.962633
2504,(22112),(23355),0.038887,0.035442,0.012192,0.313520,8.845909,0.010814,1.405077,0.922840


In [42]:
# Se filtran y recomeindan los productos basados en un producto especifico
def recommend_association_rules(stockcode, rules, n=10):
    product_rules = rules[rules['antecedents'].apply(lambda x: stockcode in x)]
    product_rules = product_rules.sort_values(by='lift', ascending=False).head(n)
    recommended_products = []
    for rule in product_rules['consequents']:
        recommended_products.extend(list(rule))
    return recommended_products

In [43]:
# Ejemplo de uso
recommendations = recommend_association_rules(stockcode="23355", rules=rules)
print(recommendations)

['23356', '22114', '22112']


# Recomendaciones Basadas en Precios y Descuentos



### Descripción: 
Si el precio ("UNITPRICE") es un factor significativo, se pueden crear recomendaciones basadas en la segmentación por precios o identificar productos frecuentemente comprados juntos en diferentes rangos de precio

### Implementación:

### Paso 1:
Se segmentan los productos por rangos de precio

### Paso 2:
Se aplican técnicas de agrupamiento para identificar productos que suelen comprarse en los mismos rangos de precio

### Paso 3:
Se recomiendan productos dentro de un rango de precio similar

In [47]:
from sklearn.cluster import KMeans

# Agrupamiento de productos basados en precios
kmeans = KMeans(n_clusters=5) # 5 grupos de precios
dataframe_retail_data['price_cluster'] = kmeans.fit_predict(dataframe_retail_data[["UNITPRICE"]])

In [48]:
# Recomendación de productos dentro del mismo grupo de precios
def recommended_based_on_price_cluster(stockcode, df, n=10):
    cluster = df[df["STOCKCODE"] == stockcode]["price_cluster"].iloc[0]
    similar_products = df[df["price_cluster"] == cluster]["STOCKCODE"].unique()
    return similar_products[:n]


In [49]:
# Ejemplo de uso
recommendations = recommended_based_on_price_cluster(stockcode="23355", df = dataframe_retail_data)
print(recommendations)

['22139' '22904' '22423' '84988' '21212' '22178' '22457' '22927' '22926'
 '22925']


: 